# Shadow Neural Radiance Fields demonstration notebook
This notebook contains the demonstration of how to render plots from a pre-trained S-NeRF model, produced using `snerf/train.py`. It is based on the use of TensorFlow-2.2.0 with a single CUDA-enabled GPU.

In [ ]:
import os
import math
import numpy as np

import tensorflow as tf
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import matplotlib.pyplot as plt
import pprint

from IPython.display import HTML, display

from snerf import data_handling
from snerf import models
from snerf import render
from snerf import train
from snerf import plots

np.set_printoptions(precision=3, suppress=True)
def_dtype = np.float32


In [ ]:
print(tf.__version__)
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)

In [ ]:
# Read config
parser = train.config_parser()
args = parser.parse_args('--config configs/068/siren_df2_full_sc_config.txt')
arg_dict = vars(args)
pprint.pprint(arg_dict)
dataset = data_handling.generate_dataset(arg_dict)

Here we visualize the image set used for training the network, as well as the corresponding light and viewing angles for the scene. We also show the Ground Truth DEM of the scene (airborne-lidar based), which is used for evaluation of the altitude estimation.

In [ ]:
plots.plot_images(arg_dict['data.train_id'], dataset['train_imgs'], dataset['train_view_dirs'], dataset['train_light_dirs'])

In [ ]:
plots.plot_view_light_directions(dataset['train_view_dirs'], dataset['train_light_dirs'])

In [ ]:
plots.plot_depth_map(dataset['depth_map'])

We load the pre-trained model and show its internal architecture, including the number of trainable parameters.

In [ ]:
model = models.load_model(f"models/068/model.npy", arg_dict)
model['model'].summary()

We need to calculate a global rescale factor for the data set based on the size of the images.

In [ ]:
# Rescale factor
arg_dict['rend.rescale'] = render.calculate_rescale_factor(dataset)

Next we can render and plot the various S-NeRF outputs : Color, depth map, sun visibility at surface, and albedo. We verify the relighting capability by performing an interpolation between a low and high solar angle. We also generate a flyover video that shows a mixture of relighting and novel view synthesis (can take time to generate).

In [ ]:
dataset_rend = render.render_dataset(dataset, model, ['rgb', 'depth', 'ret_sun', 'no_shadow'], arg_dict)

In [ ]:
plots.plot_results(dataset['train_imgs'], dataset['train_focals'], dataset_rend['train_rend'])

In [ ]:
plots.plot_results(dataset['test_imgs'], dataset['test_focals'], dataset_rend['test_rend'])

In [ ]:
plots.render_vertical_depth_comparison(model, arg_dict, dataset['depth_map'])

In [ ]:
hwf = [dataset['train_imgs'][0].shape[0], dataset['train_imgs'][0].shape[1], 617000.0/0.3/arg_dict["data.image.df"]]
light_start = np.rad2deg(dataset['train_light_dirs'][4][0,0]), np.rad2deg(dataset['train_light_dirs'][4][0,1])
light_end = np.rad2deg(dataset['train_light_dirs'][11][0,0]), np.rad2deg(dataset['train_light_dirs'][11][0,1])
view_angle=(np.pi, np.pi/2) #Vertical view : elevation = 90 deg, az = 180 deg (north up)

rets=['rgb', 'depth', 'sky', 'no_shadow']
plots.plot_light_angle_inter(model, arg_dict, hwf, light_start, light_end, view_angle, nplots=5, rets=rets)


In [ ]:
plots.render_flyover_video(f"{arg_dict['out.path']}/", model, arg_dict, hwf, light_start, light_end, rets)